**Задача:**
- сделать классификацию тональности текста с помощью BERT
- дообучить модель на предоставленном датасете
- добавить оценку качества модели на валидационном датасете после каждой эпохи обучения средствами библиотеки Evaluate (F1)
- сохранять веса модели после каждой эпохи обучения
- сохранить веса лучшей модели

In [1]:
!pip install -q datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [3]:
from typing import List, Tuple

In [4]:
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [5]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
MINIBATCH_SIZE = 124

NUM_EPOCHС = 20
LEARNING_RATE = 2e-5

In [6]:
# инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# загрузка датасета
def load_sib200_ru() -> DatasetDict:
    dataset = load_dataset('Davlan/sib200', 'rus_Cyrl')

    train_labels = set(dataset['train']['category'])
    dataset['validation'] = dataset['validation'].filter(lambda x: x['category'] in train_labels)
    dataset['test'] = dataset['test'].filter(lambda x: x['category'] in train_labels)

    return dataset

dataset = load_sib200_ru()


README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

Filter:   0%|          | 0/204 [00:00<?, ? examples/s]

In [8]:
# 3. Преобразование категорий в числовые метки
label2id = {label: i for i, label in enumerate(sorted(set(dataset['train']['category'])))}
id2label = {i: label for label, i in label2id.items()}

# Добавляем колонку 'labels', которая содержит числовые метки
def encode_labels(example):
    example['labels'] = label2id[example['category']]
    return example

dataset = dataset.map(encode_labels)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [9]:
# токенизация обучающего и валидационного датасетов
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [10]:
# инициализация метрики из evaluate
metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Используем average='macro' для многоклассовой задачи
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [11]:
# создание дата коллатора
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
# создание списка классов и словарей соответствия классов числовым меткам
num_labels = len(set(dataset['train']['category']))

In [13]:
# инициализация классификатора через AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label2id))

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#короч у меня была ошибка, нужно сделать тензоры непрерывными - так что пункт от себя
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()  # делаем типа в непрерывный тензор

In [15]:
# задаём аргументы для обучения через TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # оценка на валид после каждой эпохи
    save_strategy="epoch",        # сейв весов модели после каждой эпохи
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=NUM_EPOCHС,
    weight_decay=0.01,
    load_best_model_at_end=True,  # сейв бест модели
    metric_for_best_model="f1",   # метрика для выбора бест модели
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# создаём Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
# обучаем
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.796228,0.101894
2,No log,1.649212,0.323789
3,No log,1.463904,0.389467
4,No log,1.254900,0.609656
5,No log,1.051269,0.705151
6,No log,0.865703,0.805609
7,No log,0.723859,0.844224
8,No log,0.612872,0.839487
9,No log,0.535055,0.836880
10,No log,0.486823,0.826120


TrainOutput(global_step=120, training_loss=0.5962388356526692, metrics={'train_runtime': 583.9957, 'train_samples_per_second': 24.007, 'train_steps_per_second': 0.205, 'total_flos': 453920864102100.0, 'train_loss': 0.5962388356526692, 'epoch': 20.0})

In [23]:
# создаём пайплайн классификации текста для получения предсказаний
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0)

In [19]:
# получаем предсказания на валидационной выборке и формируем classification_report
val_predictions = trainer.predict(tokenized_datasets['validation'])
val_preds = np.argmax(val_predictions.predictions, axis=-1)
val_report = classification_report(tokenized_datasets['validation']['labels'], val_preds, target_names=list(label2id.keys()))
print("Валидационная выборка:\n", val_report)

Валидационная выборка:
                     precision    recall  f1-score   support

     entertainment       1.00      0.67      0.80         9
         geography       0.78      0.88      0.82         8
            health       1.00      0.82      0.90        11
          politics       0.87      0.93      0.90        14
science/technology       0.96      0.96      0.96        25
            sports       1.00      0.92      0.96        12
            travel       0.67      0.80      0.73        20

          accuracy                           0.87        99
         macro avg       0.90      0.85      0.87        99
      weighted avg       0.89      0.87      0.87        99



In [20]:
# получаем предсказания на тестовой выборке и формируем classification_report
test_predictions = trainer.predict(tokenized_datasets['test'])
test_preds = np.argmax(test_predictions.predictions, axis=-1)
test_report = classification_report(tokenized_datasets['test']['labels'], test_preds, target_names=list(label2id.keys()))
print("Тестовая выборка:\n", test_report)

Тестовая выборка:
                     precision    recall  f1-score   support

     entertainment       0.80      0.63      0.71        19
         geography       0.89      1.00      0.94        17
            health       0.95      0.91      0.93        22
          politics       0.97      0.93      0.95        30
science/technology       0.89      0.96      0.92        51
            sports       0.96      0.92      0.94        25
            travel       0.93      0.95      0.94        40

          accuracy                           0.92       204
         macro avg       0.91      0.90      0.90       204
      weighted avg       0.92      0.92      0.91       204



In [21]:
# пишем вывод


В данной работе была выполнена классификация тональности текста с помощью модели BERT (DeepPavlov/rubert-base-cased). Датасет был токенизирован и подготовлен для многоклассовой классификации. Модель обучалась с оценкой качества на валидационной выборке после каждой эпохи с использованием F1-метрики. Лучшие веса модели сохранялись.
Среднее значение f1 на тестовом датасете 0,92, что считается хорошим результатом в обучении нейронных сетей.

In [25]:
!tar -cf files.tar /content/results

tar: Removing leading `/' from member names
^C


In [27]:
!zip lab1.3NLP.zip /content/results

  adding: content/results/ (stored 0%)
